# 영화 평점 분석 실습

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

## 1. 영화 평점 데이터 적재 및 전처리

In [ ]:
# 사용자 데이터 읽어오기
users = pd.read_csv('data/movielens/users.dat', sep = '::', engine = 'python',
                   names = ['사용자아이디', '성별','연령','직업','지역'])
users.head()

In [ ]:
# 평점 데이터 읽어오기
ratings = pd.read_csv('data/movielens/ratings.dat', sep = '::', engine = 'python',
                   names = ['사용자아이디', '영화아이디','평점','타임스탬프'])
ratings.head()

In [ ]:
# 영화데이터 읽어오기
movies = pd.read_csv('data/movielens/movies.dat', sep = '::', engine = 'python',
                   names = ['영화아이디','영화제목','장르'], encoding = 'latin-1')
movies.head()

In [ ]:
#3개의 데이터프레임을 하나로 합치기

## 2. 보고 싶은 영화  찾기 (평점 높은 영화 찾기)

In [ ]:
# 영화들의 평점 평균을 구하여, 평점이 높은 영화 찾기 

## 3. (실습) 여자들이 좋아하는 영화 찾기 (평점이 4.0 이상이고 여성 평점의 개수가 500개 이상인 영화)

### 3-1. (심화) 여자들이 어떤 장르의 영화를 좋아하는지 찾아보자.(위에서 찾은 여성들이 좋아하는 영화들의 장르를 찾아낸 후, 어떤 장르가 많았는지 확인)

## (실습) 4. 남자와 여자의 호불호가 크게 갈리는 영화 10개를 출력 
### 전체 평점의 개수가 500개 이상인 영화만 대상으로 함.

## 5. (실습) 연령대 별로 영화 평점 분석하기
### 연령대(10대 미만, 10대, 20대, ...50대) 컬럼을 추가한 후, 연령대로 집계를 수행하여 영화별 연령대별 영화평점 구하기